In [1]:
import os
import spacy
import pandas as pd
from spacy.tokens import DocBin

FILE = 'sentiments_twitter_filtrado.csv'

PATH = os.path.join(os.path.abspath(''), f'data\{FILE}')

df = pd.read_csv(PATH)

df['sentiment'].value_counts()

df

In [ ]:
train = df.groupby('sentiment').sample(df.groupby('sentiment').size().min()-49, random_state = 25)
#test = df.groupby('sentiment').sample(frac=0.9, random_state = 9)
#train = df.groupby('sentiment').sample(random_state = 9)
test = df.drop(train.index)

print(test.index)
print(train.index)

print(train.shape, test.shape)

print(train['sentiment'].value_counts())
print(test['sentiment'].value_counts())

In [ ]:
train['tuples'] = train.apply(lambda row: (row['text'],row['sentiment']), axis=1)
train = train['tuples'].tolist()
test['tuples'] = test.apply(lambda row: (row['text'],row['sentiment']), axis=1)
test = test['tuples'].tolist()
train[0]

In [ ]:
# User function for converting the train and test dataset into spaCy document

nlp=spacy.load("pt_core_news_lg")
def document(data):
#Creating empty list called "text"
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
    elif (label=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
    else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
    #Adding the doc into the list 'text'
    text.append(doc)
  return(text)

In [ ]:
#passing the train dataset into function 'document'
train_docs = document(train)
test_docs = document(test)

#Creating binary document using DocBin function in spaCy
doc_bin_train = DocBin(docs = train_docs)
doc_bin_test = DocBin(docs = test_docs)

#Saving the binary document as train.spacy
doc_bin_train.to_disk("train.spacy")
doc_bin_test.to_disk("test.spacy")